In [11]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [12]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [13]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

# 1. DQN Agent

In [14]:
from agent import ReinforceAgent
from agent.model import ConvPolicyModel
from agent.configuration import PGConfig
from agent.exploration import LinearExploration

In [15]:
model = ConvPolicyModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=16,
    output_dim=49
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = PGConfig(
    policy_model=model,
    ckpt_path='./PG_gobang_n7',
)
agent = ReinforceAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/reinforcement_gobang_n7')

2023-05-05 08:48:44.737 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.017804622650146484, reward sum: -22
2023-05-05 08:48:48.103 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 26/50
2023-05-05 08:48:48.257 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: -0.6124191284179688, reward sum: -31
2023-05-05 08:48:48.380 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.40799808502197266, reward sum: -26
2023-05-05 08:48:48.639 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 1.5803565979003906, reward sum: -48.2
2023-05-05 08:48:48.847 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: -0.10279273986816406, reward sum: -42
2023-05-05 08:48:48.994 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.17021560668945312, reward sum: -36
2023-05-05 08:48:49.190 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.7971305847167969, reward sum: -46
2023-05-05 08:48:49.379 | INFO     | __

In [16]:
model = ConvPolicyModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=16,
    output_dim=121
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = PGConfig(
    policy_model=model,
    ckpt_path='./PG_gobang_n11',
)
agent = ReinforceAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/reinforcement_gobang_n11')

2023-05-05 09:04:21.221 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: -0.6053123474121094, reward sum: -56
2023-05-05 09:04:26.134 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 29/50
2023-05-05 09:04:26.424 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 1.6557579040527344, reward sum: -65
2023-05-05 09:04:26.669 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 3.272876739501953, reward sum: -60
2023-05-05 09:04:26.918 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 1.2101211547851562, reward sum: -57
2023-05-05 09:04:27.253 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: -0.2372283935546875, reward sum: -68
2023-05-05 09:04:27.583 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 11.437577247619629, reward sum: -75
2023-05-05 09:04:27.902 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 13.50698471069336, reward sum: -80
2023-05-05 09:04:28.210 | INFO     | __main__:do

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0